In [ ]:
# 필요한 모듈과 라이브러리를 로딩합니다.
import os 
import pandas as pd
import urllib.request
from selenium import webdriver   
import time
from selenium.webdriver.common.by import By
from datetime import datetime, date
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

In [ ]:
# 1일 주기 리스트 생성
start_date = '20120601'
end_date = '20230319'
date_list = pd.date_range(start=start_date, end=end_date, freq='d')

date_str_list = []
for date in date_list:
    date_str_list.append(date.strftime('%Y%m%d'))

# 수집 실패 날짜 리스트    
error_date = []

# 크롬드라이버 열기
driver = webdriver.Chrome() # 크롬드라이버 경로
driver.maximize_window() # 크롬창 크기 최대

# 도시 xpath 리스트 정의
xpath_city_list = [
    '/html/body/div[2]/div[1]/div/div[2]/div[3]/div[3]/form/div/div[2]/div[1]/div/div/div/div/div/ul[3]/li/ul/li[1]/ul/li[9]/a[2]',
    '/html/body/div[2]/div[1]/div/div[2]/div[3]/div[3]/form/div/div[2]/div[1]/div/div/div/div/div/ul[3]/li/ul/li[2]/ul/li[2]/a[2]',
    '/html/body/div[2]/div[1]/div/div[2]/div[3]/div[3]/form/div/div[2]/div[1]/div/div/div/div/div/ul[3]/li/ul/li[3]/ul/li[11]/a[2]',
    '/html/body/div[2]/div[1]/div/div[2]/div[3]/div[3]/form/div/div[2]/div[1]/div/div/div/div/div/ul[3]/li/ul/li[4]/ul/li[14]/a[2]',
    '/html/body/div[2]/div[1]/div/div[2]/div[3]/div[3]/form/div/div[2]/div[1]/div/div/div/div/div/ul[3]/li/ul/li[5]/ul/li/a[2]',
    '/html/body/div[2]/div[1]/div/div[2]/div[3]/div[3]/form/div/div[2]/div[1]/div/div/div/div/div/ul[3]/li/ul/li[6]/ul/li[1]/a[2]',
    '/html/body/div[2]/div[1]/div/div[2]/div[3]/div[3]/form/div/div[2]/div[1]/div/div/div/div/div/ul[3]/li/ul/li[7]/ul/li/a[2]',
    '/html/body/div[2]/div[1]/div/div[2]/div[3]/div[3]/form/div/div[2]/div[1]/div/div/div/div/div/ul[3]/li/ul/li[8]/ul/li/a[2]',
    '/html/body/div[2]/div[1]/div/div[2]/div[3]/div[3]/form/div/div[2]/div[1]/div/div/div/div/div/ul[3]/li/ul/li[9]/ul/li[2]/a[2]',
    '/html/body/div[2]/div[1]/div/div[2]/div[3]/div[3]/form/div/div[2]/div[1]/div/div/div/div/div/ul[3]/li/ul/li[11]/ul/li/a[2]',
    '/html/body/div[2]/div[1]/div/div[2]/div[3]/div[3]/form/div/div[2]/div[1]/div/div/div/div/div/ul[3]/li/ul/li[12]/ul/li[3]/a[2]',
    '/html/body/div[2]/div[1]/div/div[2]/div[3]/div[3]/form/div/div[2]/div[1]/div/div/div/div/div/ul[3]/li/ul/li[13]/ul/li[8]/a[2]',
    '/html/body/div[2]/div[1]/div/div[2]/div[3]/div[3]/form/div/div[2]/div[1]/div/div/div/div/div/ul[3]/li/ul/li[14]/ul/li[9]/a[2]',
    '/html/body/div[2]/div[1]/div/div[2]/div[3]/div[3]/form/div/div[2]/div[1]/div/div/div/div/div/ul[3]/li/ul/li[15]/ul/li[6]/a[2]',
    '/html/body/div[2]/div[1]/div/div[2]/div[3]/div[3]/form/div/div[2]/div[1]/div/div/div/div/div/ul[3]/li/ul/li[16]/ul/li[5]/a[2]',
    '/html/body/div[2]/div[1]/div/div[2]/div[3]/div[3]/form/div/div[2]/div[1]/div/div/div/div/div/ul[3]/li/ul/li[17]/ul/li[4]/a[2]'    
]

In [ ]:
# 링크 접속
link = 'https://data.kma.go.kr/data/grnd/selectAsosRltmList.do?pgmNo=36'
driver.get(link)
time.sleep(2)

# 로그인
path_login = '/html/body/div[2]/header/div[2]/div/div/ul[2]/li[2]/a'
login_button = driver.find_element(By.XPATH, path_login)
driver.execute_script("arguments[0].click();", login_button)
time.sleep(1)

# ID, PW 입력
path_id = '/html/body/div[3]/div/div/div[2]/form/div[1]/div/div[1]/input'
path_pw = '/html/body/div[3]/div/div/div[2]/form/div[1]/div/div[2]/input'
path_login_button = '/html/body/div[3]/div/div/div[2]/form/div[1]/div/button'

driver.find_element(By.XPATH, path_id).send_keys('----------')  # 개인정보라 삭제
driver.find_element(By.XPATH, path_pw).send_keys('----------')  # 개인정보라 삭제
driver.find_element(By.XPATH, path_login_button).click()
time.sleep(2)

# 분 자료 선택
path_time_select = '/html/body/div[2]/div[1]/div/div[2]/div[3]/div[3]/form/div/div[1]/dl[1]/dd/select'
driver.find_element(By.XPATH, path_time_select).click()
time.sleep(1)

path_minute = '/html/body/div[2]/div[1]/div/div[2]/div[3]/div[3]/form/div/div[1]/dl[1]/dd/select/option[text()="분 자료"]'
driver.find_element(By.XPATH, path_minute).click()
time.sleep(1)

# 지점 및 기상 정보 선택
count = 0
for i in range(1,18):
    if i == 10:
        continue
    path_zoom = '/html/body/div[2]/div[1]/div/div[2]/div[3]/div[3]/form/div/div[2]/div[1]/div/div/div/div/div/ul[3]/li/ul/li[' + str(i) + ']/a[1]'
    driver.find_element(By.XPATH, path_zoom).click()
    time.sleep(0.2)
    
    driver.find_element(By.XPATH, xpath_city_list[count]).send_keys(Keys.ENTER)
    count += 1
    
    driver.find_element(By.XPATH, path_zoom).click()
    time.sleep(0.2)

path_weather = '/html/body/div[2]/div[1]/div/div[2]/div[3]/div[3]/form/div/div[2]/div[2]/div/div/div/div/div/ul/li/a[2]'
driver.find_element(By.XPATH, path_weather).click()
time.sleep(1)

# 기간 선택
for date in date_str_list:
    driver.execute_script(f'document.querySelector("input[id=startDt_d]").removeAttribute("readonly")')
    driver.execute_script(f'document.querySelector("input[id=startDt_d]").value = "{date}"')
    time.sleep(2)
    
    driver.execute_script(f'document.querySelector("input[id=endDt_d]").removeAttribute("readonly")')
    driver.execute_script(f'document.querySelector("input[id=endDt_d]").value = "{date}"')
    time.sleep(2)

    try:
        path_submit = '/html/body/div[2]/div[1]/div/div[2]/div[3]/div[3]/form/div/div[3]/button'
        driver.find_element(By.XPATH, path_submit).click()
    except:
        error_date.append(date)
        time.sleep(60)
        continue

    element = WebDriverWait(driver, 60).until(EC.presence_of_element_located((By.XPATH, '/html/body/div[2]/div[1]/div/div[2]/div[3]/div[4]/div[1]/div/a[1]')))
    time.sleep(20)
    
    while True:
        try:
            # csv 파일 다운로드
            path_csv = '/html/body/div[2]/div[1]/div/div[2]/div[3]/div[4]/div[1]/div/a[1]'
            driver.find_element(By.XPATH, path_csv).click()
        except:
            time.sleep(30)
            continue
    
    if date == date_str_list[0]:
        time.sleep(3)
        path_method = '/html/body/div[5]/div/div/div/div/div[2]/form/ul/li[6]/input'
        driver.find_element(By.XPATH, path_method).click()
        time.sleep(2)

        path_final_submit = '/html/body/div[5]/div/div/div/div/div[2]/div/a[2]'
        driver.find_element(By.XPATH, path_final_submit).click()
    
    element = WebDriverWait(driver, 60).until(EC.presence_of_element_located((By.XPATH, '/html/body/div[2]/div[1]/div/div[2]/div[3]/div[3]/form/div/div[3]/button')))
    time.sleep(30)
    
driver.close()

In [ ]:
# 다운로드받은 csv파일 : df에 저장
count = 0
for f_name in os.listdir('./Crawling_Data'):
    if f_name.startswith('OBS_ASOS_MI_202306'):
        if count == 0:
            df = pd.read_csv(f'./Crawling_Data/{f_name}', encoding='cp949')
        else:
            df = pd.concat([df, pd.read_csv(f'./Crawling_Data/{f_name}', encoding='cp949')], ignore_index=True)
        count += 1
        
# df -> csv
df.to_csv('weather_data_v1.csv')